In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd
import skforecast

print(skforecast.__version__)

c:\Users\jaesc2\GitHub\skforecast
0.18.0


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from skforecast.model_selection._split import TimeSeriesFold, OneStepAheadFold
from skforecast.plot.plot import backtesting_gif_creator

In [17]:
y = pd.Series(np.arange(100))
# y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
cv = TimeSeriesFold(
        steps                 = 11,
        initial_train_size    = 70,
        fold_stride           = 5,
        window_size           = 6,
        differentiation       = None,
        refit                 = True,
        fixed_train_size      = False,
        gap                   = 2,
        skip_folds            = 2,
        allow_incomplete_fold = True,
        return_all_indexes    = True,
        verbose    = True,
    )
folds = cv.split(X=y, as_pandas=True)
folds

Information of folds
--------------------
Number of observations used for initial training: 70
Number of observations used for backtesting: 30
    Number of folds: 6
    Number skipped folds: 3 [1, 3, 5]
    Number of steps per fold: 11
    Number of steps to the next fold (fold stride): 5
    Number of steps to exclude between last observed data (last window) and predictions (gap): 2
    Last fold only includes 3 observations.

Fold: 0
    Training:   0 -- 69  (n=70)
    Validation: 72 -- 82  (n=11)
Fold: 1
    Fold skipped
Fold: 2
    Training:   0 -- 79  (n=80)
    Validation: 82 -- 92  (n=11)
Fold: 3
    Fold skipped
Fold: 4
    Training:   0 -- 89  (n=90)
    Validation: 92 -- 99  (n=8)
Fold: 5
    Fold skipped



,fold,train_index,last_window_index,test_index,test_index_with_gap,fit_forecaster
0,0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(64, 65, 66, 67, 68, 69)","(70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 8...","(72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82)",True
1,1,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(74, 75, 76, 77, 78, 79)","(80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 9...","(82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92)",True
2,2,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(84, 85, 86, 87, 88, 89)","(90, 91, 92, 93, 94, 95, 96, 97, 98, 99)","(92, 93, 94, 95, 96, 97, 98, 99)",True


In [7]:
y = pd.Series(np.arange(100))
y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
cv = OneStepAheadFold(
        initial_train_size    = 70,
        window_size           = 3,
        differentiation       = None,
        return_all_indexes    = False,
        verbose               = False
    )
folds = cv.split(X=y)
folds

[[0, 70], [70, 100], True]

In [9]:
# list attributes 
dir(cv)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_extract_index',
 '_print_info',
 '_repr_html_',
 '_validate_params',
 'differentiation',
 'initial_train_size',
 'return_all_indexes',
 'set_params',
 'split',
 'verbose',
 'window_size']

In [4]:
backtesting_gif_creator(
    data=y,
    cv=cv,
    plot_last_window=False,
    filename="backtesting.gif",
    fps=1
)

Information of folds
--------------------
Number of observations used for initial training: 70
Number of observations used for backtesting: 30
    Number of folds: 6
    Number skipped folds: 0 
    Number of steps per fold: 11
    Number of steps to exclude between folds (fold stride): 5
    Number of steps to exclude between last observed data (last window) and predictions (gap): 0
    Last fold only includes 5 observations.

Fold: 0
    Training:   2022-01-01 00:00:00 -- 2022-03-11 00:00:00  (n=70)
    Validation: 2022-03-12 00:00:00 -- 2022-03-22 00:00:00  (n=11)
Fold: 1
    Training:   2022-01-06 00:00:00 -- 2022-03-16 00:00:00  (n=70)
    Validation: 2022-03-17 00:00:00 -- 2022-03-27 00:00:00  (n=11)
Fold: 2
    Training:   2022-01-11 00:00:00 -- 2022-03-21 00:00:00  (n=70)
    Validation: 2022-03-22 00:00:00 -- 2022-04-01 00:00:00  (n=11)
Fold: 3
    Training:   2022-01-16 00:00:00 -- 2022-03-26 00:00:00  (n=70)
    Validation: 2022-03-27 00:00:00 -- 2022-04-06 00:00:00  (n=11)


'c:\\Users\\jaesc2\\GitHub\\skforecast\\dev\\backtesting.gif'

In [38]:
# Libraries
# ==============================================================================
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from skforecast.datasets import fetch_dataset
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection import TimeSeriesFold
from skforecast.model_selection import backtesting_forecaster
from skforecast.plot import plot_prediction_intervals, set_dark_theme

In [52]:
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "datetime"], "header": 0}
)

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 2)


In [53]:
# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data[['y']]
data = data.sort_index()
data.head(3)

,y
datetime,
1991-07-01,0.429795
1991-08-01,0.400906
1991-09-01,0.432159


In [1]:
# Create TimeSeriesFold
# ==============================================================================
end_train = '2002-01-01 23:59:00'
cv = TimeSeriesFold(
         steps                 = 10,
         initial_train_size    = len(data.loc[:end_train]),
         window_size           = 10,
         refit                 = True,
         fixed_train_size      = False,
         gap                   = 0,
         allow_incomplete_fold = True,
         verbose               = False
     )

NameError: name 'TimeSeriesFold' is not defined

In [69]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor       = LGBMRegressor(random_state=123, verbose=-1),
                 lags            = 15,
                 window_features = RollingFeatures(stats=['mean'], window_sizes=[10])
             )

cv = TimeSeriesFold(
        steps                 = 10,
        initial_train_size    = len(data.loc[:end_train]),
        fold_stride           = 15,
        refit                 = True,
        fixed_train_size      = False,
        gap                   = 0,
        allow_incomplete_fold = True
     )

metric, predictions = backtesting_forecaster(
                          forecaster    = forecaster,
                          y             = data['y'],
                          cv            = cv,
                          metric        = 'mean_squared_error',
                          n_jobs        = 'auto',
                          verbose       = False,
                          show_progress = True
                      )
predictions

  0%|          | 0/6 [00:00<?, ?it/s]

,fold,pred
2002-02-01,0,0.570082
2002-03-01,0,0.731883
2002-04-01,0,0.697942
2002-05-01,0,0.752874
2002-06-01,0,0.730697
2002-07-01,0,0.847688
2002-08-01,0,0.907662
2002-09-01,0,0.932360
2002-10-01,0,0.988048
2002-11-01,0,1.008386


In [66]:
predictions.groupby('fold').size()

fold
0     10
1     10
2     10
3     10
4     10
5     10
6     10
7     10
8     10
9     10
10    10
11    10
12    10
13    10
14     7
15     2
dtype: int64

In [65]:
metric

,mean_squared_error
0,0.008913


In [58]:
preds = [
    pd.Series(np.arange(100)) for i in range(10)
]

In [61]:
fold_labels = [
    np.repeat(i, pred.shape[0]) for i, pred in enumerate(preds)
]
fold_labels = np.concatenate(fold_labels)
fold_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,

In [49]:
folds = 10
steps = 10
fold_stride = 5

preds = []
for fold in range(folds):
    pred = pd.DataFrame(
        {
            'pred': np.arange(steps),
            'fold': [fold] * steps,
        },
        index=pd.date_range(start='2022-01-01', periods=steps, freq='D')
    )
    pred.index = pred.index + DateOffset(days=fold_stride * fold)
    preds.append(pred)

preds = pd.concat(preds)
preds.head(20)

,pred,fold
2022-01-01,0,0
2022-01-02,1,0
2022-01-03,2,0
2022-01-04,3,0
2022-01-05,4,0
2022-01-06,5,0
2022-01-07,6,0
2022-01-08,7,0
2022-01-09,8,0
2022-01-10,9,0


In [51]:
preds.loc[preds.index.duplicated(keep='first')]

,pred,fold
2022-01-06,0,1
2022-01-07,1,1
2022-01-08,2,1
2022-01-09,3,1
2022-01-10,4,1
2022-01-11,0,2
2022-01-12,1,2
2022-01-13,2,2
2022-01-14,3,2
2022-01-15,4,2


In [50]:
preds.loc[~preds.index.duplicated(keep='last')]

,pred,fold
2022-01-01,0,0
2022-01-02,1,0
2022-01-03,2,0
2022-01-04,3,0
2022-01-05,4,0
2022-01-06,0,1
2022-01-07,1,1
2022-01-08,2,1
2022-01-09,3,1
2022-01-10,4,1


In [35]:
%%timeit

preds.loc[~preds.index.duplicated(keep='last')]

95 μs ± 1.44 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [44]:
~preds.index.duplicated(keep='last')

array([ True,  True,  True,  True,  True, False, False, False, False,
       False,  True,  True,  True,  True,  True, False, False, False,
       False, False,  True,  True,  True,  True,  True, False, False,
       False, False, False,  True,  True,  True,  True,  True, False,
       False, False, False, False,  True,  True,  True,  True,  True,
       False, False, False, False, False,  True,  True,  True,  True,
        True, False, False, False, False, False,  True,  True,  True,
        True,  True, False, False, False, False, False,  True,  True,
        True,  True,  True, False, False, False, False, False,  True,
        True,  True,  True,  True, False, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [42]:
preds.index.duplicated(keep='first')

array([False, False, False, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True, False, False, False,
       False, False,  True,  True,  True,  True,  True, False, False,
       False, False, False,  True,  True,  True,  True,  True, False,
       False, False, False, False,  True,  True,  True,  True,  True,
       False, False, False, False, False,  True,  True,  True,  True,
        True, False, False, False, False, False,  True,  True,  True,
        True,  True, False, False, False, False, False,  True,  True,
        True,  True,  True, False, False, False, False, False,  True,
        True,  True,  True,  True, False, False, False, False, False,
        True,  True,  True,  True,  True, False, False, False, False,
       False])

In [47]:
preds[preds.duplicated(keep='first')]

,pred,fold


In [37]:
%%timeit

preds.sort_values("fold").loc[~preds.index.duplicated(keep='last')]

176 μs ± 5.08 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
preds.sort_values("fold").drop_duplicates(keep="last")

,pred,fold
2022-01-01,0,0
2022-01-02,1,0
2022-01-02,0,1
2022-01-03,1,1
2022-01-03,0,2
2022-01-04,1,2
2022-01-04,0,3
2022-01-05,1,3
2022-01-05,0,4
2022-01-06,1,4


In [25]:
preds.sort_values("fold").drop_duplicates(keep="last").groupby('fold').size()

fold
0    2
1    2
2    2
3    2
4    2
5    2
6    2
7    2
8    2
9    2
dtype: int64

In [29]:
out = (preds.reset_index(names='date')
           .loc[lambda df: df.groupby('date')["fold"].idxmax()]
           .set_index('date'))
out

,pred,fold
date,,
2022-01-01,0,0
2022-01-02,0,1
2022-01-03,0,2
2022-01-04,0,3
2022-01-05,0,4
2022-01-06,0,5
2022-01-07,0,6
2022-01-08,0,7
2022-01-09,0,8


In [ ]:
pd.testing.assert_frame_equal(
    preds.loc[preds.groupby(level=0)["fold"].idxmax()], 
    preds.loc[~preds.index.duplicated(keep='last')]
)

AssertionError: DataFrame are different

DataFrame shape mismatch
[left]:  (100, 2)
[right]: (55, 2)

In [25]:
results = [
    (pd.DataFrame({
        "pred": np.arange(1000),
        "fold": np.arange(1000)
    }),
    ['a', 'b', 'c', 'd'] * 5,
    ['a', 'b', 'c', 'd'] * 5,
    ['a', 'b', 'c', 'd'] * 5,
    ['a', 'b', 'c', 'd'] * 5,
    ['a', 'b', 'c', 'd'] * 5,
    ['a', 'b', 'c', 'd'] * 5,
    ['a', 'b', 'c', 'd'] * 5,
    ['a', 'b', 'c', 'd'] * 5,
    ['a', 'b', 'c', 'd'] * 5,
    ['a', 'b', 'c', 'd'] * 5,
    ['a', 'b', 'c', 'd'] * 5)
    for i in range(1000)
]

In [31]:
%%timeit

pd.concat([result[0] for result in results], axis=0)
set(chain(*[result[1] for result in results]))

30.8 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%%timeit

backtest_predictions, backtest_levels, *_ = zip(*results)
pd.concat(backtest_predictions, axis=0)
set(chain(*backtest_levels))

32.3 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
import pandas as pd

# Creamos dos DataFrames con un índice y una columna "level"
df1 = pd.DataFrame({"pred": [1, 2]}, index=[pd.Timestamp("2022-01-01"), pd.Timestamp("2022-01-02")])
df1["level"] = 0

df2 = pd.DataFrame({"pred": [3, 4]}, index=[pd.Timestamp("2022-01-01"), pd.Timestamp("2022-01-02")])
df2["level"] = 1

results = [(df1, None), (df2, None)]
results = pd.concat([result[0] for result in results], axis=0)
results

,pred,level
2022-01-01,1,0
2022-01-02,2,0
2022-01-01,3,1
2022-01-02,4,1


In [42]:


(
    pd.concat([df1, df2], axis=0)
    .insert(0, 'fold', np.array([1, 2, 3, 4]))
    .rename_axis('idx', axis=0)
    .set_index('level', append=True)
)

AttributeError: 'NoneType' object has no attribute 'rename_axis'

In [ ]:

# Tu código
backtest_predictions = (
    pd.concat([result[0] for result in results], axis=0)
    .rename_axis('idx', axis=0)
    .set_index('level', append=True)
)

print("SIN sort_index():")
print(backtest_predictions)

print("\nCON sort_index():")
print(backtest_predictions.sort_index())


In [39]:
import pandas as pd

df = pd.DataFrame({
    "date": ["2021-01", "2021-01", "2021-02", "2021-02"],
    "id": [1, 2, 1, 2],
    "value": [20, 10, 30, 40]
})
df = df.set_index("date")
display(df)

# Append 'id' to the existing index (which is RangeIndex by default)
df2 = df.set_index("id", append=True)

print(df2)

,id,value
date,,
2021-01,1,20
2021-01,2,10
2021-02,1,30
2021-02,2,40


            value
date    id       
2021-01 1      20
        2      10
2021-02 1      30
        2      40


In [43]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from skforecast.datasets import fetch_dataset
from skforecast.preprocessing import (
    reshape_series_wide_to_long,
    reshape_series_long_to_dict, 
    RollingFeatures
)
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import (
    OneStepAheadFold,
    TimeSeriesFold,
    backtesting_forecaster_multiseries,
    grid_search_forecaster_multiseries,
    random_search_forecaster_multiseries,
    bayesian_search_forecaster_multiseries
)
from skforecast.plot import set_dark_theme

In [44]:
# DataFrame wide to long
# ==============================================================================
data = fetch_dataset(name="items_sales")
display(data.head(3))

print("DataFrame wide to long:")
series_long = reshape_series_wide_to_long(data=data)
display(series_long.head(3))

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


,item_1,item_2,item_3
date,,,
2012-01-01,8.253175,21.047727,19.429739
2012-01-02,22.777826,26.578125,28.009863
2012-01-03,27.549099,31.751042,32.078922


DataFrame wide to long:


value
series_id datetime             
item_1    2012-01-01   8.253175
          2012-01-02  22.777826
          2012-01-03  27.549099

In [46]:
# Split data into train-test
# ==============================================================================
end_train = '2014-07-15 23:59:00'
series_long_train = series_long.loc[series_long.index.get_level_values('datetime') <= end_train, :].copy()
series_long_test  = series_long.loc[series_long.index.get_level_values('datetime') > end_train, :].copy()

In [49]:
# Create and train ForecasterRecursiveMultiSeries
# ==============================================================================
forecaster = ForecasterRecursiveMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 window_features    = RollingFeatures(stats=['mean', 'mean'], window_sizes=[24, 48]),
                 encoding           = 'ordinal',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

forecaster.fit(series=series_long_train, store_in_sample_residuals=True)
forecaster

╭────────────────────────────────── InputTypeWarning ──────────────────────────────────╮
│ Passing a DataFrame (either wide or long format) as `series` requires additional     │
│ internal transformations, which can increase computational time. It is recommended   │
│ to use a dictionary of pandas Series instead. For more details, see:                 │
│ https://skforecast.org/latest/user_guides/independent-multi-time-series-forecasting. │
│ html#input-data                                                                      │
│                                                                                      │
│ Category : skforecast.exceptions.InputTypeWarning                                    │
│ Location : c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:2350          │
│ Suppress : warnings.simplefilter('ignore', category=InputTypeWarning)                │
╰──────────────────────────────────────────────────────────────────────────────────────╯

============================== 
ForecasterRecursiveMultiSeries 
============================== 
Regressor: LGBMRegressor 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
Window features: ['roll_mean_24', 'roll_mean_48'] 
Window size: 48 
Series encoding: ordinal 
Series names (levels): item_1, item_2, item_3 
Exogenous included: False 
Exogenous names: None 
Transformer for series: None 
Transformer for exog: None 
Weight function included: False 
Series weights: None 
Differentiation order: None 
Training range: 
    'item_1': ['2012-01-01', '2014-07-15'], 'item_2': ['2012-01-01', '2014-07-15'],
    'item_3': ['2012-01-01', '2014-07-15'] 
Training index type: DatetimeIndex 
Training index frequency: D 
Regressor parameters: 
    {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0,
    'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1,
    'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0,
    'n_estimators': 100, 'n_jobs': None, 'num_leaves': 31, 'objective': None,
    'random_state': 123, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0,
    'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': -1} 
fit_kwargs: {} 
Creation date: 2025-09-01 16:30:27 
Last fit date: 2025-09-01 16:30:27 
Skforecast version: 0.18.0 
Python version: 3.12.11 
Forecaster id: None

In [72]:
# Backtesting multiple time series
# ==============================================================================
cv = TimeSeriesFold(
         steps              = 24,
         initial_train_size = '2014-07-15 23:59:00',  # end_train
         fold_stride        = 24,
         refit              = True
     )

metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
    forecaster            = forecaster,
    series                = series_long,
    exog                  = None,
    cv                    = cv,
    levels                = None,
    metric                = 'mean_absolute_error',
    add_aggregated_metric = True,
    return_predictors=True,
    verbose = True
)

print("Backtest metrics")
display(metrics_levels)
print("")
print("Backtest predictions")
backtest_predictions.head(4)

╭────────────────────────────────── InputTypeWarning ──────────────────────────────────╮
│ Passing a DataFrame (either wide or long format) as `series` requires additional     │
│ internal transformations, which can increase computational time. It is recommended   │
│ to use a dictionary of pandas Series instead. For more details, see:                 │
│ https://skforecast.org/latest/user_guides/independent-multi-time-series-forecasting. │
│ html#input-data                                                                      │
│                                                                                      │
│ Category : skforecast.exceptions.InputTypeWarning                                    │
│ Location : c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:2350          │
│ Suppress : warnings.simplefilter('ignore', category=InputTypeWarning)                │
╰──────────────────────────────────────────────────────────────────────────────────────╯

Information of folds
--------------------
Number of observations used for initial training: 927
Number of observations used for backtesting: 170
    Number of folds: 8
    Number skipped folds: 0 
    Number of steps per fold: 24
    Number of steps to exclude between last observed data (last window) and predictions (gap): 0
    Last fold only includes 2 observations.

Fold: 0
    Training:   2012-01-01 00:00:00 -- 2014-07-15 00:00:00  (n=927)
    Validation: 2014-07-16 00:00:00 -- 2014-08-08 00:00:00  (n=24)
Fold: 1
    Training:   2012-01-25 00:00:00 -- 2014-08-08 00:00:00  (n=927)
    Validation: 2014-08-09 00:00:00 -- 2014-09-01 00:00:00  (n=24)
Fold: 2
    Training:   2012-02-18 00:00:00 -- 2014-09-01 00:00:00  (n=927)
    Validation: 2014-09-02 00:00:00 -- 2014-09-25 00:00:00  (n=24)
Fold: 3
    Training:   2012-03-13 00:00:00 -- 2014-09-25 00:00:00  (n=927)
    Validation: 2014-09-26 00:00:00 -- 2014-10-19 00:00:00  (n=24)
Fold: 4
    Training:   2012-04-06 00:00:00 -- 2014-10-1

  0%|          | 0/8 [00:00<?, ?it/s]

Backtest metrics


,levels,mean_absolute_error
0,item_1,1.194108
1,item_2,2.595094
2,item_3,3.316836
3,average,2.368679
4,weighted_average,2.368679
5,pooling,2.368679



Backtest predictions


,level,fold,pred,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,...,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,roll_mean_24,roll_mean_48,_level_skforecast
2014-07-16,item_1,0,25.698703,25.980745,23.307307,22.609388,23.773923,25.662128,26.067678,24.367445,...,23.211437,24.326191,24.574091,25.770848,25.531300,21.526332,20.821503,23.957928,23.354549,0.0
2014-07-16,item_2,0,10.469603,10.489583,10.895833,8.100000,11.008333,11.002083,9.052083,12.670833,...,12.260417,13.177083,13.377083,11.616667,9.829167,13.883333,11.893750,11.218142,12.115842,1.0
2014-07-16,item_3,0,11.329472,9.908915,9.315334,13.028927,16.139173,10.396751,9.677730,12.199832,...,15.207243,15.332012,15.183975,16.153380,13.442053,12.734172,11.130242,12.383152,13.238411,2.0
2014-07-17,item_1,0,25.676440,25.698703,25.980745,23.307307,22.609388,23.773923,25.662128,26.067678,...,21.642111,23.211437,24.326191,24.574091,25.770848,25.531300,21.526332,24.161145,23.404512,0.0


In [59]:
backtest_predictions

,level,fold,pred
2014-07-16,item_1,0,25.698703
2014-07-16,item_2,0,10.469603
2014-07-16,item_3,0,11.329472
2014-07-17,item_1,0,25.676440
2014-07-17,item_2,0,11.036070
...,...,...,...
2014-12-31,item_2,14,17.146157
2014-12-31,item_3,14,17.986318
2015-01-01,item_1,14,19.936686
2015-01-01,item_2,14,17.049874


In [60]:
backtest_predictions = (
    backtest_predictions
    .rename_axis('idx', axis=0)
    .set_index('level', append=True)
)
backtest_predictions


fold       pred
idx        level                  
2014-07-16 item_1     0  25.698703
           item_2     0  10.469603
           item_3     0  11.329472
2014-07-17 item_1     0  25.676440
           item_2     0  11.036070
...                 ...        ...
2014-12-31 item_2    14  17.146157
           item_3    14  17.986318
2015-01-01 item_1    14  19.936686
           item_2    14  17.049874
           item_3    14  17.943596

[984 rows x 2 columns]

In [69]:

(
    backtest_predictions
    .loc[~backtest_predictions.index.duplicated(keep='last')]
).loc[pd.to_datetime('2014-07-16') + DateOffset(days=11)]

level      item_3
fold            0
pred     9.576969
Name: 2014-07-27 00:00:00, dtype: object

In [ ]:
cv = TimeSeriesFold(
         steps              = 24,
         initial_train_size = '2014-07-15 23:59:00',  # end_train
         fold_stride        = 12,
         refit              = True
     )